# 1차, 2차 학습 시도 실패했다.
# 모델 자체가 한계 있는 건 아닐까? 
# 모델을 바꿔보자. 

In [1]:
import keras 
from keras.applications import xception
from keras import layers
from keras import preprocessing

base_model = keras.applications.xception.Xception(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False,
)  # Do not include the ImageNet classifier at the top.

base_model.trainable = False

inputs = keras.Input(shape=(150, 150, 3))

norm_layer = keras.layers.BatchNormalization()
x = norm_layer(inputs)

x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.5)(x)  # Regularize with dropout
outputs = keras.layers.Dense(20)(x)
model = keras.Model(inputs, outputs)

model.summary()

2022-02-17 15:38:35.970758: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-17 15:38:35.971001: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 xception (Functional)       (None, 5, 5, 2048)        20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 20)                40980     
                                                                 
Total params: 20,902,460
Trainable params: 40,980
N

In [2]:
# 이미지 데이터 전처리 # 데이터 증식 사용
import keras
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255) # 검증데이터는 증식하지 않는다.

train_generator = train_datagen.flow_from_directory(
    '/Users/kibeomkim/Desktop/flickr/train', 
    target_size=(150, 150),
    batch_size=100, 
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    '/Users/kibeomkim/Desktop/flickr/test',
    target_size=(150,150),
    batch_size=50,
    class_mode='categorical'
)

Found 7993 images belonging to 20 classes.
Found 1999 images belonging to 20 classes.


In [30]:
# 가장 기본적인 3가지 콜백 
callbacks_list = [

    #과대적합 발생 시 조기 스톱 콜백
    keras.callbacks.EarlyStopping(
        monitor='val_categorical_accuracy', # 모델 검증 손실 모니터링
        patience= 20), # 20에폭동안 과대적합 관찰
    
    # 최고의 가중치 자동 저장 콜백
    keras.callbacks.ModelCheckpoint(
        '/Users/kibeomkim/Desktop/models_saved/my_model.h5', 
        monitor = 'val_loss', 
        save_best_only = True), 

    # 학습률 자동 조정 콜백
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.1,
        patience=5
    ),

    # 텐서보드 
    keras.callbacks.TensorBoard(
        log_dir = '/Users/kibeomkim/Desktop/my_log_dir',
        histogram_freq = 1, # 1에포크마다 층 출력의 히스토그램 기록
        embeddings_freq = 1
    )
]

In [84]:
import keras.optimizers

model.compile(
    optimizer=keras.optimizers.adam_v2.Adam(), 
    loss = keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics = [keras.metrics.CategoricalAccuracy()]
)

history = model.fit_generator(
    train_generator,
    steps_per_epoch = 80, 
    epochs=15,
    validation_data = validation_generator, 
    validation_steps=40, 
    verbose=2,
    callbacks=callbacks_list 
)

Epoch 1/15


2022-02-17 12:26:10.365633: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-17 12:28:43.899119: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


80/80 - 171s - loss: 2.5442 - categorical_accuracy: 0.2317 - val_loss: 2.2189 - val_categorical_accuracy: 0.3397 - lr: 0.0010 - 171s/epoch - 2s/step
Epoch 2/15
80/80 - 174s - loss: 2.1429 - categorical_accuracy: 0.3451 - val_loss: 2.1385 - val_categorical_accuracy: 0.3557 - lr: 0.0010 - 174s/epoch - 2s/step
Epoch 3/15
80/80 - 169s - loss: 2.0243 - categorical_accuracy: 0.3771 - val_loss: 2.1109 - val_categorical_accuracy: 0.3552 - lr: 0.0010 - 169s/epoch - 2s/step
Epoch 4/15
80/80 - 172s - loss: 1.9434 - categorical_accuracy: 0.3982 - val_loss: 2.0895 - val_categorical_accuracy: 0.3627 - lr: 0.0010 - 172s/epoch - 2s/step
Epoch 5/15
80/80 - 158s - loss: 1.8921 - categorical_accuracy: 0.4111 - val_loss: 2.0716 - val_categorical_accuracy: 0.3767 - lr: 0.0010 - 158s/epoch - 2s/step
Epoch 6/15
80/80 - 161s - loss: 1.8436 - categorical_accuracy: 0.4269 - val_loss: 2.0791 - val_categorical_accuracy: 0.3732 - lr: 0.0010 - 161s/epoch - 2s/step
Epoch 7/15
80/80 - 162s - loss: 1.8159 - categorica

# 미세조정

In [85]:
base_model.trainable = True
model.summary()

model.compile(
    optimizer=keras.optimizers.adam_v2.Adam(1e-5),  # Low learning rate
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.CategoricalAccuracy()],
)


history = model.fit_generator(
    train_generator,
    steps_per_epoch = 80, 
    epochs=200,
    validation_data = validation_generator, 
    validation_steps=40, 
    verbose=2,
    callbacks=callbacks_list 
)

Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_46 (InputLayer)       [(None, 150, 150, 3)]     0         
                                                                 
 batch_normalization_97 (Bat  (None, 150, 150, 3)      12        
 chNormalization)                                                
                                                                 
 xception (Functional)       (None, 5, 5, 2048)        20861480  
                                                                 
 global_average_pooling2d_14  (None, 2048)             0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dropout_12 (Dropout)        (None, 2048)              0         
                                                                 
 dense_13 (Dense)            (None, 20)                409

2022-02-17 13:07:33.502734: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-17 13:12:58.425582: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


80/80 - 344s - loss: 1.5614 - categorical_accuracy: 0.5147 - val_loss: 2.0397 - val_categorical_accuracy: 0.3912 - lr: 1.0000e-05 - 344s/epoch - 4s/step
Epoch 2/200
80/80 - 408s - loss: 1.4674 - categorical_accuracy: 0.5316 - val_loss: 2.0342 - val_categorical_accuracy: 0.4007 - lr: 1.0000e-05 - 408s/epoch - 5s/step
Epoch 3/200
80/80 - 399s - loss: 1.4188 - categorical_accuracy: 0.5485 - val_loss: 2.0229 - val_categorical_accuracy: 0.3997 - lr: 1.0000e-05 - 399s/epoch - 5s/step
Epoch 4/200
80/80 - 403s - loss: 1.3628 - categorical_accuracy: 0.5662 - val_loss: 2.0290 - val_categorical_accuracy: 0.4022 - lr: 1.0000e-05 - 403s/epoch - 5s/step
Epoch 5/200
80/80 - 327s - loss: 1.3190 - categorical_accuracy: 0.5826 - val_loss: 2.0430 - val_categorical_accuracy: 0.4037 - lr: 1.0000e-05 - 327s/epoch - 4s/step
Epoch 6/200
80/80 - 327s - loss: 1.2699 - categorical_accuracy: 0.5945 - val_loss: 2.0494 - val_categorical_accuracy: 0.4072 - lr: 1.0000e-05 - 327s/epoch - 4s/step
Epoch 7/200
80/80 - 32

KeyboardInterrupt: 

# Xception 모델도 40% 이상 검증 성능 향상되지 않았다. 

# 소규모 컨브넷 만들어보자. 

In [10]:
from keras import layers 
from keras import models 

model = models.Sequential()
model.add(layers.Conv2D(32, (3,3),activation='relu', input_shape=(150,150,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(20, activation='softmax'))

In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 17, 17, 128)     

In [12]:
from keras import optimizers
model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizers.adam_v2.Adam(learning_rate=0.001),
    metrics=['acc']
)

In [13]:
model.fit_generator(
    train_generator, 
    steps_per_epoch=80,
    epochs=20,
    validation_data = validation_generator,
    validation_steps=40,
    verbose=2,
    callbacks=callbacks_list
)

2022-02-17 14:39:04.097210: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-02-17 14:39:04.255970: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/20


2022-02-17 14:39:42.778494: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


80/80 - 44s - loss: 2.7822 - acc: 0.1468 - val_loss: 2.7695 - val_acc: 0.1526 - lr: 0.0010 - 44s/epoch - 545ms/step
Epoch 2/20
80/80 - 42s - loss: 2.5629 - acc: 0.2141 - val_loss: 2.5590 - val_acc: 0.2226 - lr: 0.0010 - 42s/epoch - 528ms/step
Epoch 3/20
80/80 - 42s - loss: 2.4609 - acc: 0.2427 - val_loss: 2.4528 - val_acc: 0.2376 - lr: 0.0010 - 42s/epoch - 531ms/step
Epoch 4/20
80/80 - 42s - loss: 2.3812 - acc: 0.2669 - val_loss: 2.4346 - val_acc: 0.2441 - lr: 0.0010 - 42s/epoch - 526ms/step
Epoch 5/20
80/80 - 42s - loss: 2.3176 - acc: 0.2852 - val_loss: 2.4010 - val_acc: 0.2526 - lr: 0.0010 - 42s/epoch - 529ms/step
Epoch 6/20
80/80 - 46s - loss: 2.2855 - acc: 0.2966 - val_loss: 2.4305 - val_acc: 0.2611 - lr: 0.0010 - 46s/epoch - 575ms/step
Epoch 7/20
80/80 - 43s - loss: 2.2281 - acc: 0.3085 - val_loss: 2.3754 - val_acc: 0.2706 - lr: 0.0010 - 43s/epoch - 540ms/step
Epoch 8/20
80/80 - 46s - loss: 2.1792 - acc: 0.3288 - val_loss: 2.3344 - val_acc: 0.2801 - lr: 0.0010 - 46s/epoch - 574ms/

# Xception 모델로 재시도 

In [22]:
import keras 
from keras.applications import xception
from keras import layers
from keras import preprocessing

base_model = keras.applications.xception.Xception(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False,
)  # Do not include the ImageNet classifier at the top.

base_model.trainable = False

inputs = keras.Input(shape=(150, 150, 3))

norm_layer = keras.layers.BatchNormalization()
x = norm_layer(inputs)

x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.7)(x)  # Regularize with dropout
outputs = keras.layers.Dense(20)(x)
model = keras.Model(inputs, outputs)

model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 150, 150, 3)]     0         
                                                                 
 batch_normalization_31 (Bat  (None, 150, 150, 3)      12        
 chNormalization)                                                
                                                                 
 xception (Functional)       (None, 5, 5, 2048)        20861480  
                                                                 
 global_average_pooling2d_6   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_6 (Dropout)         (None, 2048)              0         
                                                                 
 dense_6 (Dense)             (None, 20)                4098

In [23]:
# 이미지 데이터 전처리 # 데이터 증식 사용
import keras
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.5,
    zoom_range=[0.8, 2.0],
    horizontal_flip=True,
    vertical_flip = True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255) # 검증데이터는 증식하지 않는다.

train_generator = train_datagen.flow_from_directory(
    '/Users/kibeomkim/Desktop/flickr/train', 
    target_size=(150, 150),
    batch_size=5, 
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    '/Users/kibeomkim/Desktop/flickr/test',
    target_size=(150,150),
    batch_size=5,
    class_mode='categorical'
)

Found 7993 images belonging to 20 classes.
Found 1999 images belonging to 20 classes.


In [24]:
# 가장 기본적인 3가지 콜백 
callbacks_list = [

    #과대적합 발생 시 조기 스톱 콜백
    keras.callbacks.EarlyStopping(
        monitor='val_categorical_accuracy', # 모델 검증 손실 모니터링
        patience= 20), # 20에폭동안 과대적합 관찰
    
    # 최고의 가중치 자동 저장 콜백
    keras.callbacks.ModelCheckpoint(
        '/Users/kibeomkim/Desktop/models_saved/my_model.h5', 
        monitor = 'val_loss', 
        save_best_only = True), 

    # 학습률 자동 조정 콜백
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.1,
        patience=10
    ),

    # 텐서보드 
    keras.callbacks.TensorBoard(
        log_dir = '/Users/kibeomkim/Desktop/my_log_dir',
        histogram_freq = 1, # 1에포크마다 층 출력의 히스토그램 기록
        embeddings_freq = 1
    )
]

In [25]:
import keras.optimizers

model.compile(
    optimizer=keras.optimizers.adam_v2.Adam(), 
    loss = keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics = [keras.metrics.CategoricalAccuracy()]
)

history = model.fit_generator(
    train_generator,
    steps_per_epoch = 1590, 
    epochs=15,
    validation_data = validation_generator, 
    validation_steps= 399, 
    #verbose=2,
    callbacks=callbacks_list 
)

Epoch 1/15


2022-02-17 16:41:19.884921: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-17 16:42:54.754333: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1000/1000 - 109s - loss: 3.0020 - categorical_accuracy: 0.1800 - val_loss: 2.4486 - val_categorical_accuracy: 0.2680 - lr: 0.0010 - 109s/epoch - 109ms/step
Epoch 2/15
1000/1000 - 110s - loss: 2.8055 - categorical_accuracy: 0.2274 - val_loss: 2.4083 - val_categorical_accuracy: 0.3020 - lr: 0.0010 - 110s/epoch - 110ms/step
Epoch 3/15
1000/1000 - 139s - loss: 2.7206 - categorical_accuracy: 0.2570 - val_loss: 2.4967 - val_categorical_accuracy: 0.2633 - lr: 0.0010 - 139s/epoch - 139ms/step
Epoch 4/15
1000/1000 - 120s - loss: 2.7093 - categorical_accuracy: 0.2614 - val_loss: 2.4827 - val_categorical_accuracy: 0.2973 - lr: 0.0010 - 120s/epoch - 120ms/step
Epoch 5/15
1000/1000 - 111s - loss: 2.6852 - categorical_accuracy: 0.2709 - val_loss: 2.4216 - val_categorical_accuracy: 0.3113 - lr: 0.0010 - 111s/epoch - 111ms/step
Epoch 6/15
1000/1000 - 105s - loss: 2.6497 - categorical_accuracy: 0.2729 - val_loss: 2.4531 - val_categorical_accuracy: 0.2987 - lr: 0.0010 - 105s/epoch - 105ms/step
Epoch 7/1

# 미세조정 

In [26]:
base_model.trainable = True
#model.summary()

model.compile(
    optimizer=keras.optimizers.adam_v2.Adam(1e-5),  # Low learning rate
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.CategoricalAccuracy()],
)


history = model.fit_generator(
    train_generator,
    steps_per_epoch = 1590, 
    epochs=200,
    validation_data = validation_generator, 
    validation_steps=399, 
    #verbose=2,
    callbacks=callbacks_list 
)

Epoch 1/200


2022-02-17 17:09:12.021406: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1590/1590 [==============================] - ETA: 0s - loss: 2.1689 - categorical_accuracy: 0.3421

2022-02-17 17:13:08.719141: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1590/1590 [==============================] - 257s 158ms/step - loss: 2.1689 - categorical_accuracy: 0.3421 - val_loss: 2.0722 - val_categorical_accuracy: 0.3729 - lr: 1.0000e-05
Epoch 2/200
1590/1590 [==============================] - 248s 156ms/step - loss: 1.9834 - categorical_accuracy: 0.3846 - val_loss: 1.9980 - val_categorical_accuracy: 0.3784 - lr: 1.0000e-05
Epoch 3/200
1590/1590 [==============================] - 253s 159ms/step - loss: 1.8674 - categorical_accuracy: 0.4207 - val_loss: 1.9540 - val_categorical_accuracy: 0.4040 - lr: 1.0000e-05
Epoch 4/200
1590/1590 [==============================] - 251s 158ms/step - loss: 1.8053 - categorical_accuracy: 0.4270 - val_loss: 1.9121 - val_categorical_accuracy: 0.4180 - lr: 1.0000e-05
Epoch 5/200
1590/1590 [==============================] - 249s 156ms/step - loss: 1.7413 - categorical_accuracy: 0.4497 - val_loss: 1.8884 - val_categorical_accuracy: 0.4115 - lr: 1.0000e-05
Epoch 6/200
1590/1590 [==============================] - 248s 

KeyboardInterrupt: 

In [28]:
train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=120,
    width_shift_range=0.8,
    height_shift_range=0.8,
    shear_range=0.8,
    zoom_range=[0.1, 0.5],
    horizontal_flip=True,
    vertical_flip = True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255) # 검증데이터는 증식하지 않는다.

train_generator = train_datagen.flow_from_directory(
    '/Users/kibeomkim/Desktop/flickr/train', 
    target_size=(150, 150),
    batch_size=5, 
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    '/Users/kibeomkim/Desktop/flickr/test',
    target_size=(150,150),
    batch_size=5,
    class_mode='categorical'
)

Found 7993 images belonging to 20 classes.
Found 1999 images belonging to 20 classes.


In [32]:
model.compile(
    optimizer=keras.optimizers.adam_v2.Adam(1e-6),  # Low learning rate
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.CategoricalAccuracy()],
)


history = model.fit_generator(
    train_generator,
    steps_per_epoch = 1590, 
    epochs=200,
    validation_data = validation_generator, 
    validation_steps=399, 
    #verbose=2,
    callbacks=callbacks_list 
)

Epoch 1/200


2022-02-17 19:29:57.949264: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1590/1590 [==============================] - ETA: 0s - loss: 2.4793 - categorical_accuracy: 0.2329

2022-02-17 19:33:53.680289: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1590/1590 [==============================] - 255s 158ms/step - loss: 2.4793 - categorical_accuracy: 0.2329 - val_loss: 1.8941 - val_categorical_accuracy: 0.4246 - lr: 1.0000e-06
Epoch 2/200
1590/1590 [==============================] - 249s 156ms/step - loss: 2.4680 - categorical_accuracy: 0.2378 - val_loss: 1.8880 - val_categorical_accuracy: 0.4306 - lr: 1.0000e-06
Epoch 3/200
1590/1590 [==============================] - 249s 156ms/step - loss: 2.4653 - categorical_accuracy: 0.2393 - val_loss: 1.8941 - val_categorical_accuracy: 0.4286 - lr: 1.0000e-06
Epoch 4/200
1590/1590 [==============================] - 243s 153ms/step - loss: 2.4647 - categorical_accuracy: 0.2389 - val_loss: 1.8948 - val_categorical_accuracy: 0.4326 - lr: 1.0000e-06
Epoch 5/200
1590/1590 [==============================] - 242s 152ms/step - loss: 2.4442 - categorical_accuracy: 0.2494 - val_loss: 1.8910 - val_categorical_accuracy: 0.4311 - lr: 1.0000e-06
Epoch 6/200
1590/1590 [==============================] - 242s 

KeyboardInterrupt: 

# 시도: 과대적합 나타나기 시작할 때 학습 수동으로 끊고 데이터 증식 조건 바꿈. --> 새로운 조건 하에서 증식된 데이터를 모델이 새 데이터로 인지할 걸로 예상함. 
# 증식으로 클래스별 데이터 수를 늘리면 과대적합도 억제하고 일반화 성능도 끌어올릴 수 있겠다고 예상함. 
# 결과: 과대적합은 꽤 잘 억제한 듯 보이나, 일반화 성능 42~43% 대에서 더 이상 증가하지 않고 박스권에 갇힘. 
# 4차 시도 실패 